<a href="https://colab.research.google.com/github/lucaskydelima/Optimization-with-Python-Pyomo/blob/main/S5Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyomo
! pip install cplex -q

In [5]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

In [18]:
from IPython.core.interactiveshell import Integer
from pyomo.core.base.objective import minimize

# Defining the model
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize = ['City1', 'City2', 'City3', 'City4', 'City5'])
model.j = pyo.Set(initialize = ['City1', 'City2', 'City3', 'City4', 'City5'])

model.ii = pyo.Set(initialize = ['City2', 'City3', 'City4', 'City5'])

#Params
C = pd.read_excel('S5P3_Data.xlsx', sheet_name='TSP', header=0, index_col=0, usecols='A:F', nrows=5)

# Decision Variable
model.x = pyo.Var(model.i, model.j, domain=pyo.Binary)
x = model.x

# Auxiliary Variable
model.u = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
u = model.u

# Objective Function
def Objective_rule(model):
      return sum(sum(C[i][j]*x[i,j] for j in model.j) for i in model.i)

model.obj = pyo.Objective(rule=Objective_rule, sense=pyo.minimize)

"""
The local constraints allow for subtours that don’t visit
all n cities.
"""
def Constraint_1(model, j):
  return sum(x[i,j] for i in model.i) == 1

model.const1 = pyo.Constraint(model.j, rule=Constraint_1)

def Constraint_2(model, i):
  return sum(x[i,j] for j in model.j) == 1

model.const2 = pyo.Constraint(model.i, rule=Constraint_2)

"""
https://how-to.aimms.com/Articles/332/332-Miller-Tucker-Zemlin-formulation.html
https://phabi.ch/2021/09/19/tsp-subtour-elimination-by-miller-tucker-zemlin-constraint/

"""
# Subtour Elimination Constraints (Miller-Tucker-Zemlin formulation)
def Constraint_3(model, i,j):
  if i != j:
    "ui - uj + N*x[i,j] <= N - 1"
    "uj > ui || uj >= ui + 1"
    return u[i] - u[j] + 5*x[i,j] <= 4
  else:
    return u[i] - u[j] == 0

model.const13 = pyo.Constraint(model.ii, model.ii, rule = Constraint_3)

# Solve
solver = pyo.SolverFactory('cplex_direct')
results = solver.solve(model)
print(results)
for i in model.i:
  for j in model.j:
    print('Salesman goes from', i, 'to', j, '=', x[i,j]())



Problem: 
- Name: 
  Lower bound: 668.0
  Upper bound: 668.0
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 30
  Number of binary variables: 25
  Number of integer variables: 5
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: 1
Solver: 
- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.0077397823333740234
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Salesman goes from City1 to City1 = 0.0
Salesman goes from City1 to City2 = 0.0
Salesman goes from City1 to City3 = 0.0
Salesman goes from City1 to City4 = 0.0
Salesman goes from City1 to City5 = 1.0
Salesman goes from City2 to City1 = 0.0
Salesman goes from City2 to City2 = 0.0
Salesman goes from City2 to City3 = 0.0
Salesman goes from City2 to City4 = 1.0
Salesman goes from City2 to City5 = 0.0
Salesman goes from City3 to City1 = 1.0
Salesman goes from City3 to City2 = 0.0
Salesman goes from City3 to City3 = 0.0
Salesman